In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132590")
exp = Experiment(workspace=ws, name="udacity-project-hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "newcompute1234"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
### YOUR CODE HERE ### 
parameter_space={
    "C": uniform(0,1),
    "max_iter" : choice(10,20,50,100,150)
}
ps =RandomParameterSampling(parameter_space)

# Specify a Policy
highpolicy = BanditPolicy(slack_factor=0.3)### YOUR CODE HERE ###

#if "training" not in os.listdir():
#    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
#from azureml.core.script_run_config import ScriptRunConfig
#est = ScriptRunConfig(source_directory="./", script= "train.py", compute_target=cpu_cluster )
est = SKLearn(source_directory="./", entry_script="train.py",compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4, 
                                     policy=highpolicy, 
                                     estimator=est)

In [ ]:
#HDrun.get_best_run_by_primary_metric()

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
#from azureml.train.hyperdrive import HyperDriveRun
HDrun = exp.submit(hyperdrive_config)
HDrun.wait_for_completion(show_output=True)
#HPrun.get_metrics()
#RunDetails(HDrun).show()


In [ ]:
RunDetails(HDrun).show()

In [ ]:
HDrun.get_best_run_by_primary_metric().get_metrics()

In [ ]:
import joblib
# Get your best run and save the model from that run.
bestHDrun = HDrun.get_best_run_by_primary_metric()
bestHDmodel= bestHDrun.register_model(model_name="bestHDrun", model_path="./outputs/HyperdriveModel.joblib", tags=bestHDrun.get_metrics())
#joblib.dump(bestHDrun, 'HyperdriveModel.joblib')

 
### YOUR CODE HERE ###

In [ ]:
print(bestHDmodel)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
webpath = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(webpath)
automldataset = pd.to_pandas

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data= pd.concat([x_train.reset_index(drop=True), y_train], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_settings={
    "experiment_timeout_hours" : 0.25,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "enable_onyx_compatible_model" : True
}

automl_config = AutoMLConfig(
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=4,
    **automl_settings)

In [ ]:
# Submit your automl run
exp_aml = Experiment(workspace=ws, name="udacity-project-aml")
automl_run = exp_aml.submit(automl_config)
automl_run.wait_for_completion()

#RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###
best_run_aml = automl_run.get_best_child()
joblib.dump(best_run_aml, 'amlRun_1.joblib') 